In [3]:
16000*2.178218

34851.488000000005

In [2]:
from pathlib import Path
import numpy as np, plotly.express as px
h = 500

ar = np.array([[float(j) for j in i.split()] for i in Path('graph').read_text().replace('\n\n', '\n').replace('\n\n', '\n').splitlines() if i])
ar = ar[:,0]

ema = 0.93
for i in range(1, len(ar)):
    ar[i] = ar[i-1] * ema + (1-ema)*ar[i]

ar = ar - ar.mean()
px.line(ar, width=1000, height=h).show()

In [ ]:
import numpy as np

# A raw sound wave, for example, a sine wave
# In practice, this would be data read from a WAV file or recording device
# The values typically range from -1 to 1 for a float-based format.
raw_sound_wave = ar
raw_sound_wave -=raw_sound_wave.mean()

# To calculate the RMS:
# 1. Square all the values
squared_wave = raw_sound_wave ** 2
# 2. Find the mean (average) of the squared values
mean_squared = np.mean(squared_wave)
# 3. Take the square root of the mean
rms = np.sqrt(mean_squared)

print(f"RMS value: {20 * np.log10(rms / 4095)}")


In [ ]:
def ema(ar, window=11):
    df = pd.DataFrame(ar)
    result = []
    n = len(ar)
    half = window // 2
    for i in range(n):
        # dynamically shrink window near edges
        start = max(0, i - half)
        end = min(n, i + half + 1)
        result.append(df.iloc[start:end, 0].mean())
    return np.array(result)

In [106]:
from pathlib import Path
import numpy as np, plotly.express as px
h = 500

ar = np.array([[float(j) for j in i.split()] for i in Path('graph').read_text().replace('\n\n', '\n').replace('\n\n', '\n').splitlines() if i])
new_sound = ema(ar[:,1], 30)
# px.scatter(x=ar[:,0], y=new_sound, title='rpm vs sound').show()
# px.scatter(x=ar[:,0], y=ar[:,2], title='rpm vs lift').show()
# px.scatter(x=ar[:,2], y=new_sound, title='lift vs sound').show()
rpm = ar[:,0].copy()
lift = ar[:,2].copy()

no_ring = {'sound': new_sound, 'rpm': rpm, 'lift': lift}

In [122]:
from pathlib import Path
import numpy as np, plotly.express as px
h = 500

ar = np.array([[float(j) for j in i.split()] for i in Path('ring prop').read_text().replace('\n\n', '\n').replace('\n\n', '\n').splitlines() if i])[3:]
new_sound = ema(ar[:,1], 30)
rpm = ar[:,0].copy()
lift = ar[:,2].copy()

ring = {'sound': new_sound, 'rpm': rpm, 'lift': lift}

In [123]:
px.line({'ring': ring['lift'], 'no ring': no_ring['lift']}, title='lift').show()
px.line({'ring': ring['rpm'], 'no ring': no_ring['rpm']}, title='rpm').show()
px.line({'ring': ring['sound'], 'no ring': no_ring['sound']}, title='sound').show()

In [ ]:
from pathlib import Path
import numpy as np, plotly.express as px, pandas as pd
h = 500

diff = 20
labels = {0:'angle', 1:'info->rotate_angle_step', 2:'xamp', 3:'info->xamp_step'}
ar = np.array([[float(j) for j in i.split()] for i in Path('graph').read_text().replace('\n\n', '\n').replace('\n\n', '\n').splitlines() if i])



def plot(arr):
    arr = np.array(arr)
    fig = px.line(arr, height=300)
    fig.update_layout(
        xaxis=dict(range=[0, len(arr)-1]),
        yaxis=dict(range=[arr.min(), arr.max()]),
        margin=dict(l=0, r=0, t=10, b=0)
    )
    fig.show()

def quan(arr: list, per=0.05, lr=0.02, max_iter=40):

    n = len(arr)
    target = n * (1 - per)

    low, high = min(arr), max(arr)
    thres = (high - low) * per + low  # initial guess ~per percentile
    print(np.quantile(arr, per))
    for _ in range(max_iter):
        # how many elements are above current threshold?
        count = sum(1 for i in arr if i > thres)

        # error = how far off we are from target
        error = count - target
        if abs(error) < 20:  # good enough (within 1 element)
            break

        print(round(thres, 3), round(count/len(arr), 4), round(error, 2), round(lr, 2))

        # gradient = sign of error (since F(t) is monotonic decreasing)
        grad = 1 if error > 0 else -1

        # update threshold
        thres += error * lr# * (high - low)

        # shrink learning rate for stability
        lr *= 0.95

    return thres


def count_blades(arr, derivative_jump=10, ema=0.95, quantile=.95):

    print('original:')
    plot(arr)

    print(f'clipped {quantile}:')
    top = np.quantile(arr, quantile)
    bottom = np.quantile(arr, 1-quantile)
    arr = [top if i > top else i if i > bottom else bottom for i in arr]
    plot(arr)

    print(f'averaged {ema} ema')
    for i in range(1, len(arr)):
        arr[i] = arr[i-1] * ema + (1-ema)*arr[i]
    plot(arr)

    print(f'derivative jump: {derivative_jump}')
    for i in range(len(arr)-derivative_jump):
        arr[i] = arr[i+derivative_jump] - arr[i]
    arr = arr[:-derivative_jump]
    plot(arr)

    print('clipped below zero')
    arr = [i if i > 0 else 0 for i in arr]
    plot(arr)

    quantile -= .05
    top = np.quantile(arr, quantile)
    print(f'quantile {quantile} {top}')
    arr = [i if i < top else top for i in arr]
    plot(arr)

    print('binary')
    middle = (max(arr) + min(arr)) * .5
    arr = [i > middle for i in arr]
    plot(arr)

    count = 0
    for i in range(1, len(arr)-1):
        if not arr[i-1] and arr[i]:
            count += 1
    print(f"counted: {count}")

    return count

count_blades(ar[:,0])

In [ ]:
1/(4095.0 * 0.707106781)

In [ ]:
1/(5000*25/1e6)
20/(4592*25/60e6)
15*8